In [168]:
import scipy
import scipy.io
import numpy as np
from scipy.linalg import norm
from scipy.sparse import csr_matrix
from scipy.linalg import norm
import pickle
import time
from collections import defaultdict
import json

import matplotlib.pyplot as plt

In [84]:
results = defaultdict(dict)

# Portfolio optimization

In [5]:
# alpha policies
def alpha_standard(k):
    return 2 / (k + 1)

def alpha_icml(k, Gap, hess, s, x):
    e = hess.dot((s - x).T).T.dot((s - x).T) ** 0.5
    t = Gap / (e * (Gap + e))
    beta = alpha_standard(k)
    return min(beta, t)

$$g(x) = -\sum_{t=1}^n \log(r_t^T x)$$ 
$$\min g(x),\ x_i \ge 0,\ \sum_{t = 1}^n x_t = 1$$

In [6]:
def portfolio(R, x):
    """
        R -- object matrix (N x n)
        x -- weights (n)
    """    
    return -np.sum(np.log(R @ x))

$$\nabla g(x) = -\sum_{t = 1}^n \frac{r_t}{r_t^T x}$$

In [7]:
def grad_portfolio_old(R, x):
    """
        R -- object matrix (N x n)
        x -- weights (n)
    """
    return -np.sum(R / (R @ x).reshape(-1, 1), axis=0)

def grad_portfolio(R, x):
    """
        R -- object matrix (N x n)
        x -- weights (n)
    """
    denom = R.dot(x)
    return -R.T.dot(1 / denom)

$$\nabla^2 g(x) = \sum_{t = 1}^n \frac{r_t r_t^T}{(r_t^T x)^2}$$

In [8]:
def hess_portfolio_old(R, x):
    """
        R -- object matrix (N x n)
        x -- weights (n)
    """
    dtype = R.dtype
    result = np.zeros([n, n], dtype=dtype)
    for i in range(N):
        result += R[i].reshape(-1, 1) @ R[i].reshape(1, -1)
        result /= (R[i] @ x) ** 2
    return result

def hess_portfolio(R, x):
    """
        R -- object matrix (N x n)
        x -- weights (n)
    """
    dtype = R.dtype
    Rx = R @ x
    Z = R / Rx.reshape(-1, 1)
    return np.einsum('ij,ik->jk', Z, Z, dtype=dtype)

In [9]:
# %%time
# foo = hess_portfolio(R, np.ones(n, dtype=np.float32) / n)

In [10]:
# %%time
# bar = hess_portfolio_old(R, np.ones(n, dtype=np.float32) / n)

In [11]:
# foo

In [12]:
# bar

In [70]:
def frank_wolfe_portfolio(R, 
                          x_0=None,
                          alpha_policy='standard',
                          max_iter=50000,
                          eps=0.001,
                          print_every=100,
                          dtype=np.float64,
                          debug_info=False):
    """
        R -- object matrix (N x n)
        x_0 -- starting point (1 x n)
        alpha_policy -- name of alpha changing policy function
        eps -- epsilon
        print_every -- print results every print_every steps
    """
    x_0 = np.ones(R.shape[1]) / R.shape[1] if not x_0 else x_0
    R = R.astype(dtype)
    x_0 = x_0.astype(dtype)
    
    x = x_0
    x_hist = []
    alpha_hist = []
    Gap_hist = []
    Q_hist = []
    time_hist = []
    grad_hist = []
    print('********* Algorithm starts *********')
    for k in range(1, max_iter + 1):
        start_time = time.time()
        Q = portfolio(R, x)

        grad_f = grad_portfolio(R, x) # 1 x n
        
        grad_min = np.min(grad_f)
        s = np.array([el == grad_min for el in grad_f], dtype=dtype)
        Gap = grad_f @ (x - s)
        
        if alpha_policy == 'standard':
            alpha = alpha_standard(k)
        elif alpha_policy == 'icml':
            hessian = hess_portfolio(R, x)
            alpha = alpha_icml(k, Gap, hessian, s, x)
        alpha = dtype(alpha)
            
        # filling history
        x_hist.append(x)
        alpha_hist.append(alpha)
        Gap_hist.append(Gap)
        Q_hist.append(Q)
        grad_hist.append(grad_f)    
        
        x = (1 - alpha) * x + alpha * s
        
        criterion = norm(x - x_hist[-1])
        if criterion <= eps * max(1, norm(x_hist[-1])):
            print('Convergence achieved!')
            print(f'iter = {k}, stepsize = {alpha}, crit = {criterion}, Q={Q}')
            break
            
        if k % print_every == 0 or k == 1:
            if not debug_info:
                print(f'iter = {k}, stepsize = {alpha}, criterion = {criterion}, Q={Q}')
            else:
                print(k)
                print(f's = {np.nonzero(s)}')
                print(f'Gap = {Gap}')
                print(f'alpha = {alpha}')
                print(f'criterion = {criterion}')
                x_nz = x[np.nonzero(x)[0]]
                print(x.dtype, R.dtype, Q.dtype, grad_f.dtype, grad_min.dtype, s.dtype, type(Gap), type(alpha), type(criterion))
                print(f'x non zero: {list(zip(x_nz, np.nonzero(x)[0]))}\n')
        time_hist.append(time.time() - start_time)
    return x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist

## 800

In [80]:
data = scipy.io.loadmat('./data/syn_1000_800_10_50.mat')
R = data['W']
N, n = R.shape

In [81]:
N, n

(1000, 800)

In [90]:
with open('data/1000_800_sol.pckl', 'rb') as f:
    sol = pickle.load(f)

Standard

In [83]:
%%time
x, \
alpha_hist, \
Gap_hist, \
Q_hist, \
time_hist, \
grad_hist = frank_wolfe_portfolio(R, 
                                  eps=1e-4, 
                                  max_iter=50000, 
                                  alpha_policy='standard', 
                                  print_every=100)

********* Algorithm starts *********
iter = 1, stepsize = 1.0, criterion = 0.9993748045653342, Q=0.056350094467775266
iter = 100, stepsize = 0.019801980198019802, criterion = 0.01439972987144384, Q=-4.5709985836724645
iter = 200, stepsize = 0.009950248756218905, criterion = 0.010643115038563125, Q=-4.571361486807065
iter = 300, stepsize = 0.006644518272425249, criterion = 0.00474717343950455, Q=-4.571366371263135
iter = 400, stepsize = 0.004987531172069825, criterion = 0.005325926023236898, Q=-4.571405551488257
iter = 500, stepsize = 0.003992015968063872, criterion = 0.0028443917899086406, Q=-4.571383013624775
iter = 600, stepsize = 0.0033277870216306157, criterion = 0.0027309115621038586, Q=-4.571399337896671
iter = 700, stepsize = 0.0028530670470756064, criterion = 0.0020328296268182625, Q=-4.5713975889606
iter = 800, stepsize = 0.0024968789013732834, criterion = 0.0017757431317911353, Q=-4.571411103432629
iter = 900, stepsize = 0.0022197558268590455, criterion = 0.002368474816092024

In [86]:
results[800]['standard'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
    'grad_hist': grad_hist
}

In [92]:
error = norm(results[800]['standard']['x'] - sol) / max(1, norm(sol))
results[800]['standard']['error'] = error
error

0.00016788759341065064

ICML

In [87]:
%%time
x, \
alpha_hist, \
Gap_hist, \
Q_hist, \
time_hist, \
grad_hist = frank_wolfe_portfolio(R, 
                                  eps=1e-4, 
                                  max_iter=50000, 
                                  alpha_policy='icml', 
                                  print_every=100)

********* Algorithm starts *********
iter = 1, stepsize = 0.4966381523767165, criterion = 0.4963276564711698, Q=0.056350094467775266
iter = 100, stepsize = 0.0077612352109741245, criterion = 0.007476288898381706, Q=-4.561672285385955
iter = 200, stepsize = 0.0062920156756810725, criterion = 0.005176060091226538, Q=-4.566811953886289
iter = 300, stepsize = 0.003720398289094894, criterion = 0.003055330690062332, Q=-4.568407811784301
iter = 400, stepsize = 0.0017153760564116564, criterion = 0.0016471490991843889, Q=-4.5691863748334205
iter = 500, stepsize = 0.0012289791831255796, criterion = 0.0013116188349367604, Q=-4.5696459287562465
iter = 600, stepsize = 0.0018159054715261112, criterion = 0.001489809081690829, Q=-4.569946541534114
iter = 700, stepsize = 0.0013659869302551928, criterion = 0.0011203432617918909, Q=-4.570159265020136
iter = 800, stepsize = 0.0014551665508736255, criterion = 0.001193665911943958, Q=-4.5703184088386415
iter = 900, stepsize = 0.0007602929079030464, criterio

In [88]:
results[800]['icml'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
    'grad_hist': grad_hist
}

In [95]:
error = norm(results[800]['icml']['x'] - sol) / max(1, norm(sol))
results[800]['icml']['error'] = error
error

0.00014503993153316166

## 1200

In [154]:
n = 1200
data = scipy.io.loadmat('./data/syn_1000_1200_10_50.mat')
R = data['W']
N, n = R.shape

In [155]:
N, n

(1000, 1200)

In [156]:
with open('data/1000_1200_sol.pckl', 'rb') as f:
    sol = pickle.load(f)

standard

In [54]:
%%time
x, \
alpha_hist, \
Gap_hist, \
Q_hist, \
time_hist, \
grad_hist = frank_wolfe_portfolio(R, 
                                  eps=1e-4, 
                                  max_iter=50000, 
                                  alpha_policy='standard', 
                                  print_every=100)

********* Algorithm starts *********
iter = 1, stepsize = 1.0, criterion = 0.9995832464915899, Q=0.0803034692757601
iter = 100, stepsize = 0.019801980198019802, criterion = 0.02337764439344425, Q=-5.11870619823452
iter = 200, stepsize = 0.009950248756218905, criterion = 0.003469138327959084, Q=-5.118834764907804
iter = 300, stepsize = 0.006644518272425249, criterion = 0.00228260401139096, Q=-5.1189300049281945
iter = 400, stepsize = 0.004987531172069825, criterion = 0.005648456283809649, Q=-5.1190063635426375
iter = 500, stepsize = 0.003992015968063872, criterion = 0.004661311656013507, Q=-5.119006304956166
iter = 600, stepsize = 0.0033277870216306157, criterion = 0.0011270008918897263, Q=-5.1190106632526025
iter = 700, stepsize = 0.0028530670470756064, criterion = 0.0009676653382835549, Q=-5.119006966735721
iter = 800, stepsize = 0.0024968789013732834, criterion = 0.0008478915798291289, Q=-5.119009173003672
iter = 900, stepsize = 0.0022197558268590455, criterion = 0.002512297351294069

In [98]:
results[1200]['standard'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
    'grad_hist': grad_hist
}

In [102]:
error = norm(results[n]['standard']['x'] - sol) / max(1, norm(sol))
results[1200]['standard']['error'] = error
error

0.00022855299436283144

ICML

In [163]:
%%time
x, \
alpha_hist, \
Gap_hist, \
Q_hist, \
time_hist, \
grad_hist = frank_wolfe_portfolio(R, 
                                  eps=1e-4, 
                                  max_iter=50000, 
                                  alpha_policy='icml', 
                                  print_every=100)

********* Algorithm starts *********
iter = 1, stepsize = 0.5183535191932658, criterion = 0.5181374935455453, Q=0.0803034692757601
iter = 100, stepsize = 0.005400723599906357, criterion = 0.006368114093045444, Q=-5.108508702893242
iter = 200, stepsize = 0.003061515733192035, criterion = 0.003468958165963421, Q=-5.114263960466085
iter = 300, stepsize = 0.0011238792120157781, criterion = 0.0013119822210430464, Q=-5.115927223041931
iter = 400, stepsize = 0.004987531172069825, criterion = 0.0016903655454745163, Q=-5.116740086802039
iter = 500, stepsize = 0.0014373272923815486, criterion = 0.001627479033520891, Q=-5.117224915587389
iter = 600, stepsize = 0.0033277870216306157, criterion = 0.001126210118466312, Q=-5.117534508922107
iter = 700, stepsize = 0.0005109017296010407, criterion = 0.000596396561487173, Q=-5.117756758459911
iter = 800, stepsize = 0.0007815230533574133, criterion = 0.0008845215302749417, Q=-5.117921405945598
iter = 900, stepsize = 0.0022197558268590455, criterion = 0.0

In [164]:
results[1200]['icml'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
    'grad_hist': grad_hist
}

In [166]:
error = norm(results[1200]['icml']['x'] - sol) / max(1, norm(sol))
results[1200]['icml']['error'] = error
error

0.00022855299436283144

## 1500

In [126]:
data = scipy.io.loadmat('./data/syn_1000_1500_10_50.mat')
R = data['W']
N, n = R.shape

In [127]:
N, n

(1000, 1500)

In [128]:
with open('data/1000_1500_sol.pckl', 'rb') as f:
    sol = pickle.load(f)

Standard

In [69]:
%%time
x, \
alpha_hist, \
Gap_hist, \
Q_hist, \
time_hist, \
grad_hist = frank_wolfe_portfolio(R, 
                                  eps=1e-4, 
                                  max_iter=50000, 
                                  alpha_policy='standard', 
                                  print_every=100)

********* Algorithm starts *********
iter = 1, stepsize = 1.0, criterion = 0.9996666110925848, Q=0.021289312869133274
iter = 100, stepsize = 0.019801980198019802, criterion = 0.015590493586394838, Q=-4.474600766473991
iter = 200, stepsize = 0.009950248756218905, criterion = 0.007813498992139708, Q=-4.474989724821573
iter = 300, stepsize = 0.006644518272425249, criterion = 0.006802815462519957, Q=-4.475070486745722
iter = 400, stepsize = 0.004987531172069825, criterion = 0.0038855048239829397, Q=-4.475087166814085
iter = 500, stepsize = 0.003992015968063872, criterion = 0.0031140721574401424, Q=-4.475078809126474
iter = 600, stepsize = 0.0033277870216306157, criterion = 0.002785263993325362, Q=-4.475089793438819
iter = 700, stepsize = 0.0028530670470756064, criterion = 0.002220118499531042, Q=-4.4750917786757505
iter = 800, stepsize = 0.0024968789013732834, criterion = 0.0019435268992091543, Q=-4.475086191560047
iter = 900, stepsize = 0.0022197558268590455, criterion = 0.002451422671810

In [130]:
results[1500]['standard'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
    'grad_hist': grad_hist
}

In [131]:
error = norm(results[1500]['standard']['x'] - sol) / max(1, norm(sol))
results[1500]['standard']['error'] = error
error

6.319718274801206e-05

ICML 

In [73]:
%%time
x, \
alpha_hist, \
Gap_hist, \
Q_hist, \
time_hist, \
grad_hist = frank_wolfe_portfolio(R, 
                                  eps=1e-4, 
                                  max_iter=50000, 
                                  alpha_policy='icml', 
                                  print_every=100)

********* Algorithm starts *********
iter = 1, stepsize = 0.463056500269208, criterion = 0.46290212236851186, Q=0.021289312869133274
iter = 100, stepsize = 0.011521761315972325, criterion = 0.009687633303622135, Q=-4.464671696688804
iter = 200, stepsize = 0.006550107264641202, criterion = 0.005109366945297007, Q=-4.470150701554282
iter = 300, stepsize = 0.00268728804871834, criterion = 0.0024800012168575583, Q=-4.471872073749786
iter = 400, stepsize = 0.0028148949335326102, criterion = 0.0023568696579499377, Q=-4.472701757437687
iter = 500, stepsize = 0.0016223340222052914, criterion = 0.0016581175948639977, Q=-4.473192890773646
iter = 600, stepsize = 0.0019084770302515445, criterion = 0.0014849487689423365, Q=-4.47351547412401
iter = 700, stepsize = 0.0016952551908507383, criterion = 0.001318962098247747, Q=-4.473746351819814
iter = 800, stepsize = 0.0011551726986775413, criterion = 0.0009662610782766826, Q=-4.473916800991374
iter = 900, stepsize = 0.0011954497225324945, criterion = 0

In [140]:
results[1500]['icml'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
    'grad_hist': grad_hist
}

In [141]:
error = norm(results[1500]['icml']['x'] - sol) / max(1, norm(sol))
results[1500]['icml']['error'] = error
error

6.319718274801206e-05

# Save results

In [171]:
with open('results/results_dict.pckl', 'wb') as f:
    pickle.dump(results, f)